In [1]:
## Import the classifier and other stuff needed to run
import classifier
import imp
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_colwidth', -1)

C:\Users\Michael's  Laptop\AppData\Roaming\Python\Python27\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


IOError: [Errno 2] No such file or directory: "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\word2vec_attb2b-20190404130509.model"

In [37]:
# Read in the csv that contains the chats as well as filling any empty chats with ''
test_df = pd.read_csv(r"C:\Users\Michael's  Laptop\Documents\XSELL\attb2b_unique_test_chats_20190416.csv", encoding='utf-8')
test_df.message_text = test_df.message_text.fillna('')

In [38]:
#define function to replace useless topics
def replace_topics(df, replace_topics, replace_value='None', topic_col='Topic'):
    for replace_topic in replace_topics:
        df.loc[df[topic_col] == replace_topic, topic_col] = replace_value
    return df
# replace useless topics with value as specific in function replace_topics which should be 'none'
useless_topics = ['Security', 'Residential', 'Directv', 'Uverse', 'Moving', 'Business Cell Service', 'Hotspot', 'non', 'TV']
test_df = replace_topics(test_df, useless_topics)
# make a copy of df get around groupby issue not showing entire chat
all_df = test_df
# stop classifiying after 10 customer messages
stop_classifier=10
test_df = test_df[test_df.role=='C'].groupby('engagement_id').head(stop_classifier)

In [39]:
# reload the classifier so as to get the newest regex changes. 
reload(classifier)
classifier_dict = {}
#Search for where topic isn't null, add it to the classifier_dict
for i in test_df.loc[test_df.Topic.notnull()].itertuples():
    classifier_dict[str(i.engagement_id)]=[str(i.Topic)]
#Search of any engagement id's that exist but isnt in classifier_dict and adds them as 'None' 
# - helps with issue where chats that weren't annotated as none till bottom and only looking at top 10
for i in test_df["engagement_id"].unique():
    if i not in classifier_dict.keys():
        classifier_dict[str(i)]=["None"]
        
#Create new column  called 'pred' and fill it with nothing
test_df["pred"] = np.nan
# go through classifier_dict and classify each message, storing only the first topic annotated 
# (just like the classifier would work)
for key in classifier_dict.keys():
    chat_df = test_df.loc[test_df["engagement_id"]==key]
    for message in chat_df["message_text"]:
        predicted_topic = classifier.predict(message)
        try:
            classifier_dict[key][1]
            next
        except:
            test_df.loc[((test_df["message_text"])==message.encode('utf-8'))&((test_df["engagement_id"])==key.encode('utf-8')),["pred"]] = predicted_topic
            classifier_dict[key] = [classifier_dict[key][0],predicted_topic]

In [40]:
#y_true are all the ground truth topics,  y_pred are the predicted topics using the classifier
y_true = [item[0] for item in classifier_dict.values()]
y_pred = [item[1] for item in classifier_dict.values()]
print classification_report(y_true,y_pred)

               precision    recall  f1-score   support

       Bundle       0.63      0.63      0.63       178
Business Line       0.36      0.65      0.47        92
        Fiber       0.92      0.78      0.85        78
     Internet       0.78      0.75      0.76       388
          New       0.46      0.53      0.49       115
         None       0.71      0.64      0.67       667

    micro avg       0.67      0.67      0.67      1518
    macro avg       0.64      0.66      0.64      1518
 weighted avg       0.69      0.67      0.67      1518



In [41]:
#Use the below to write the dataframe to 
test_df.to_csv("C:\Users\Michael's  Laptop\Documents\XSELL\live_data_classified_lenckosstyle2.csv", index=False, encoding='utf-8')

In [42]:
print test_df["Topic"].dropna().size
print test_df["pred"].dropna().size
#There's a discrepancy because there are 10 messages cutoff and sometimes none is below that cutoff

1288
1516


In [43]:
test_df.loc[(test_df["Topic"]=="New")&(test_df["pred"]!="New")]

,message_text,role,engagement_id,Topic,pred
158,"xxx Lazio Circle, Debary, FL 32713",C,"""724054857838524208""",New,NaN
949,I have 3 lines at Verizon we all have s9plus phones looking to open up a business account,C,"""724335565256975842""",New,Business Line
1797,"I am trying to get services. It is telling me that DSL is available, but not giving me the option for a phone line.",C,"""723491155761985448""",New,Business Line
2854,"Hi, I'm starting a small business from home and would like to find out about att business package",C,"""724054190568153500""",New,Bundle
3411,Comcast requires a 60 day notice prior to cancellation of services. How to I coordinate installation and starting services with AT&T?,C,"""724054479660789730""",New,None
5416,"xxxx W COLONIAL DR STE 1, ORL, FL 32804",C,"""723491636608052852""",New,NaN
7536,Landlord says only AT&T is available,C,"""745165969848295876""",New,NaN
8116,I need a quote,C,"""723491751461201430""",New,None
8925,Hello! I'm looking for the package that will suit my need.Can you help me with that?,C,"""723491909266648325""",New,None
10752,what field am I not completing correctly?,C,"""732781138891453038""",New,None
